In [84]:
import math
import nbimporter
import CoolProp.CoolProp as CP

In [85]:
def f_Churchill(V_dot: float, D_h: float) -> float:
    """
    Determine the Darcy friction factor using the Churchill correlation.
    
    Parameters:
    V_dot - volume flow rate [ft/min]
    D_h - Diameter of the duct [units must match those of e]

    Returns:
    The Darcy friction factor [dimensionless]
    """
    
    rho = CP.PropsSI("D", "P", 101325, "T", 297, "Air") * 0.06243 # density of air. lbm/ft^3
    mu = CP.PropsSI("V", "P", 101325, "T", 297, "Air") * 0.671968994813 # viscosity, lbm-s/ft
    m = V_dot * rho / 60 # lbm/s
    Re = 4 * m / (math.pi * D_h * mu) # Reynolds number [dimensionless]
    e = 0.004 #ft

    A = (-2.457 * math.log(((7 / Re) ** 0.9) + (0.27 * (e / D_h)))) ** 16
    B = (37530 / Re) ** 16
    f_Churchill = 8 * (((8 / Re) ** 12) + (1 / ((A + B) ** 1.5))) ** (1 / 12)
    
    return f_Churchill

In [86]:
def P_vel(V_dot: float, Ac: float) -> float:
    """
    Determine the velocity pressure.
    
    Parameters:
    V_dot - Volume flow rate [ft^3/min]
    Ac - area [ft^2]
    
    
    Returns:
    velocity pressure [dimensionless]
    """
    rho = CP.PropsSI("D", "P", 101325, "T", 297, "Air") * 0.06243 # density of air. lbm/ft^3 
    v = V_dot / Ac / 60 # ft/s
    
    return   rho * v ** 2/2 * 0.005974451 # change to  inH2O
    

### Problem 11.5. Fume hood pressure drop
 Two identical exhaust hoods are installed in a manufacturing plant to remove the gases from a welding process. Each hood is required to remove 500 cfm from the work area. A filter is placed in the duct to prevent solid particles from damaging the fan. A pressure drop measurement across the filter is used to determine when to replace the filter. The loss coefficient for a new filter is 5.5 and the loss coefficient for each fume hood is 3.3, and the fan is sized so that the system flow for a new filter is 1200 cfm.\
a. Determine the pressure loss (inches water) between the inlet to the fume hoods and the inlet to the fan for the design conditions.\
b. Determine the value of the pressure drop that would necessitate a change of filter.

1) Information given:

In [87]:
P_atm = 14.7 # atmosphere pressure
T_z = 75 
D_1 = 9/12 # convert diameter from in to feet
V_dot_1 = 500 # cfm
V_dot_2 = V_dot_1 * 2
L_1 = 15 # ft
D_2 = 12/12 # convert diameter from in to feet
L_2 = 10 #ft
K_hood = 3.3 #loss coefficient for each fume hood
K_tee = 1.34
K_filter  = 5.5

Ac_1 = math.pi * (D_1)**2/4  # area of duct, ft^2
Ac_2 = math.pi * (D_2)**2/4 # area of duct, ft^2

2. Calculate the air viscosity and density at standard air pressure and 75F drybulb temperature

In [88]:
mu = CP.PropsSI("V", "P", 101325, "T", 297, "Air") * 0.671968994813 # viscosity, lbm-s/ft
rho = CP.PropsSI("D", "P", 101325, "T", 297, "Air") * 0.06243 # density of air. lbm/ft^3 
print ("viscosity =", mu,'[lbm-s/ft]')
print ("density =", rho, '[lbm/ft^3]')

viscosity = 1.235917638724685e-05 [lbm-s/ft]
density = 0.07422407519055603 [lbm/ft^3]


3. Calculate the pressure drop

In [97]:
f_1 = f_Churchill(V_dot_1, D_1)
f_2 = f_Churchill(V_dot_2, D_2)
P_vel_1 = P_vel(V_dot_1, Ac_1)
P_vel_2 = P_vel(V_dot_2, Ac_2)
DP_1 = K_hood  * P_vel_1
DP_2 = (f_1 * (L_1/D_1)) * P_vel_1
DP_3 = K_tee * P_vel_1
DP_4 = (f_2 * (L_2/D_2)) * P_vel_2
DP_filter = K_filter * P_vel_2

print ('Hood pressure drop', f"{DP_1:.3f}",'inH2O')
print ('Duct (before Tee) pressure drop', f"{DP_2:.3f}",'inH2O')
print ('Tee pressure drop', f"{DP_3:.3f}",'inH2O')
print ('Duct (after Tee) pressure drop', f"{DP_4:.3f}",'inH2O')
print ('DP_filter', f"{DP_filter:.3f}",'inH2O')

Hood pressure drop 0.260 inH2O
Duct (before Tee) pressure drop 0.051 inH2O
Tee pressure drop 0.106 inH2O
Duct (after Tee) pressure drop 0.029 inH2O
DP_filter 0.549 H2O
